# NBA Stats Crawler
- NBA stats official site: https://stats.nba.com/teams/boxscores/
- Selenium, locating elements: https://selenium-python.readthedocs.io/locating-elements.html
- Reference: https://ithelp.ithome.com.tw/articles/10185964
    - Example: /Users/ltc/Projects/notes/crawler.ipynb

## Setup

In [1]:
import time
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# User input
outPathRaw = './Z_raw/'
outPathArranged = './Z_arranged/'
outFile = './nbaGamePair.csv'
isAppend = 1
seasonYear = '2017-18'
seasonType = 'Playoffs'
dateStart = '2018-04-01'
pageNum = 5
loadTime = 2

# Create path if necessary
if not os.path.exists(outPathRaw):
    os.makedirs(outPathRaw)
if not os.path.exists(outPathArranged):
    os.makedirs(outPathArranged)

# Date of crawling
dateOfCrawl = time.strftime("%Y-%m-%d-h%Hm%Ms%S", time.localtime())

# URL selection
prefix = 'https://stats.nba.com/teams/boxscores/' + '?Season='
url = {
    'Preseason': prefix + seasonYear + '&SeasonType=Pre%20Season',
    'RegularSeason': prefix + seasonYear + '&SeasonType=Regular%20Season',
    'Playoffs': prefix + seasonYear + '&SeasonType=Playoffs',
    'All-Star': prefix + seasonYear + '&SeasonType=All%20Star'
}

# Open browser
driver = webdriver.Chrome(executable_path='./webDriver/chromedriver')
driver.get(url[seasonType])

# Wait for loading the web
time.sleep(loadTime)

## Scrape Maximum Page Information

In [2]:
# Scrape the maximum page informtion
content = []
curSoup = BeautifulSoup(driver.page_source, 'html.parser')
info = curSoup.findAll('select', {'class':'stats-table-pagination__select'})
for x in info[-1]:
    content.append(x.get_text())
maxPage = int(content[-1])

## Load Pages

In [3]:
# Load pages
pages = []
pageNum = min(pageNum, maxPage)
pageSel = Select(driver.find_element_by_class_name('stats-table-pagination__select'))
for i in range(1, pageNum+1):
    # Pagination (Mimic a browser that clicks "next page".)
    pageSel.select_by_value('number:'+str(i))
    # Wait for loading the web
    time.sleep(loadTime)
    # Capture current page
    pages.append(driver.page_source)
# Shutdown the browser
driver.quit()
print('----- Note: %d of %d page(s) scraped. -----' %(pageNum, maxPage))

----- Note: 4 of 4 page(s) scraped. -----


## Raw Box Acquisition

In [4]:
# Scrape box from each page and create DataFrame
soups = [BeautifulSoup(page, 'html.parser') for page in pages]
df_box = None
for soup in soups:
    # Scrape boxes' title
    boxTitle = []
    for item in soup.findAll('thead'):
        boxTitle.append(item.get_text())
    # Store boxes' title as list
    title = [x for x in boxTitle[0].split('\n') if x != '' and x != 'Season']
    
    # Scrape boxes
    boxes = []
    for item in soup.findAll('tr', {'data-ng-repeat':'(i, row) in page track by row.$hash'}):
        boxes.append(item.get_text())
    # Store boxes as list
    boxData = []
    for i in range(len(boxes)//2):
        boxData.append([x for x in boxes[i].split('\n') if x != ''])
    for i in range(len(boxData)):
        for j in range(4, len(boxData[i])):
            if j == 8 or j == 11 or j == 14:
                boxData[i][j] = round(float(boxData[i][j])*0.01, 3)
            else:
                boxData[i][j] = int(boxData[i][j])
    
    # Create/Append data frame
    if df_box is None:
        df_box = pd.DataFrame(boxData, columns=title)
    else:
        df_box = df_box.append(pd.DataFrame(boxData, columns=title), ignore_index=True)

In [5]:
# Date reformation and selection
date = []
df_box.rename(columns = {'Game\xa0Date':'Date'}, inplace=True)
for x in df_box['Date']:
    date.append(x[-4:] + '-' + x[0:2] + '-' + x[3:5])
df_box['Date'] = date
df_box = df_box.loc[(df_box.Date >= dateStart), :].reset_index(drop=True)
df_box

,Team,Match Up,Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,CLE,CLE vs. GSW,2018-06-08,L,240,85,30,87,0.345,8,...,0.680,17,27,44,21,5,5,11,22,-23
1,GSW,GSW @ CLE,2018-06-08,W,240,108,39,86,0.453,14,...,1.000,10,34,44,25,7,13,10,24,23
2,GSW,GSW @ CLE,2018-06-06,W,240,110,42,81,0.519,9,...,0.895,6,31,37,27,6,5,11,20,8
3,CLE,CLE vs. GSW,2018-06-06,L,240,102,40,92,0.435,9,...,0.765,15,32,47,20,6,4,14,18,-8
4,GSW,GSW vs. CLE,2018-06-03,W,240,122,47,82,0.573,15,...,0.619,7,34,41,28,3,8,12,25,19
5,CLE,CLE @ GSW,2018-06-03,L,240,103,37,90,0.411,9,...,0.769,16,26,42,25,9,4,11,15,-19
6,GSW,GSW vs. CLE,2018-05-31,W,265,124,46,90,0.511,13,...,0.950,4,34,38,31,10,6,8,18,10
7,CLE,CLE @ GSW,2018-05-31,L,265,114,44,99,0.444,10,...,0.727,19,34,53,18,5,3,12,18,-10
8,GSW,GSW @ HOU,2018-05-28,W,240,101,39,80,0.488,16,...,0.500,11,31,42,25,9,7,17,22,9
9,HOU,HOU vs. GSW,2018-05-28,L,240,92,36,90,0.400,7,...,0.591,17,27,44,17,13,3,12,17,-9


In [6]:
# Save raw box as .csv
df_box.to_csv(outPathRaw + dateOfCrawl + '_' + seasonYear + '_' + seasonType + '.csv', encoding='utf-8', index=False, float_format='%.3f')
df_box

,Team,Match Up,Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,CLE,CLE vs. GSW,2018-06-08,L,240,85,30,87,0.345,8,...,0.680,17,27,44,21,5,5,11,22,-23
1,GSW,GSW @ CLE,2018-06-08,W,240,108,39,86,0.453,14,...,1.000,10,34,44,25,7,13,10,24,23
2,GSW,GSW @ CLE,2018-06-06,W,240,110,42,81,0.519,9,...,0.895,6,31,37,27,6,5,11,20,8
3,CLE,CLE vs. GSW,2018-06-06,L,240,102,40,92,0.435,9,...,0.765,15,32,47,20,6,4,14,18,-8
4,GSW,GSW vs. CLE,2018-06-03,W,240,122,47,82,0.573,15,...,0.619,7,34,41,28,3,8,12,25,19
5,CLE,CLE @ GSW,2018-06-03,L,240,103,37,90,0.411,9,...,0.769,16,26,42,25,9,4,11,15,-19
6,GSW,GSW vs. CLE,2018-05-31,W,265,124,46,90,0.511,13,...,0.950,4,34,38,31,10,6,8,18,10
7,CLE,CLE @ GSW,2018-05-31,L,265,114,44,99,0.444,10,...,0.727,19,34,53,18,5,3,12,18,-10
8,GSW,GSW @ HOU,2018-05-28,W,240,101,39,80,0.488,16,...,0.500,11,31,42,25,9,7,17,22,9
9,HOU,HOU vs. GSW,2018-05-28,L,240,92,36,90,0.400,7,...,0.591,17,27,44,17,13,3,12,17,-9


## Arranged Box Acquisition

In [7]:
# Drop 'MIN'
df_box = df_box.drop(columns=['MIN'])

In [8]:
# Create 'Score' and 'Home/Away' columns
score = []
homeAway = []
for team, match, pts, pm in zip(df_box['Team'], df_box['Match\xa0Up'], df_box['PTS'], df_box['+/-']):
    # 'Score'
    oppo = match[-3:]
    score.append(oppo + str(pts-pm) + '-' + str(pts) + team)
    # 'Home/Away'
    if '@' in match:
        homeAway.append('Away')
    else:
        homeAway.append('Home')
df_box['Score'] = score
df_box['Home/Away'] = homeAway
df_box = df_box.drop(columns=['Match\xa0Up', '+/-'])
df_box

,Team,Date,W/L,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,Score,Home/Away
0,CLE,2018-06-08,L,85,30,87,0.345,8,27,0.296,...,17,27,44,21,5,5,11,22,GSW108-85CLE,Home
1,GSW,2018-06-08,W,108,39,86,0.453,14,38,0.368,...,10,34,44,25,7,13,10,24,CLE85-108GSW,Away
2,GSW,2018-06-06,W,110,42,81,0.519,9,26,0.346,...,6,31,37,27,6,5,11,20,CLE102-110GSW,Away
3,CLE,2018-06-06,L,102,40,92,0.435,9,31,0.290,...,15,32,47,20,6,4,14,18,GSW110-102CLE,Home
4,GSW,2018-06-03,W,122,47,82,0.573,15,36,0.417,...,7,34,41,28,3,8,12,25,CLE103-122GSW,Home
5,CLE,2018-06-03,L,103,37,90,0.411,9,27,0.333,...,16,26,42,25,9,4,11,15,GSW122-103CLE,Away
6,GSW,2018-05-31,W,124,46,90,0.511,13,36,0.361,...,4,34,38,31,10,6,8,18,CLE114-124GSW,Home
7,CLE,2018-05-31,L,114,44,99,0.444,10,37,0.270,...,19,34,53,18,5,3,12,18,GSW124-114CLE,Away
8,GSW,2018-05-28,W,101,39,80,0.488,16,39,0.410,...,11,31,42,25,9,7,17,22,HOU92-101GSW,Away
9,HOU,2018-05-28,L,92,36,90,0.400,7,44,0.159,...,17,27,44,17,13,3,12,17,GSW101-92HOU,Home


Preferred column order:<br/>
Team, Date, W/L, Home/Away, Score, FG%, FGM, FGA, 3P%, 3PM, 3PA, FT%, FTM, FTA, REB, OREB, DREB, AST, STL, BLK, TOV, PF, PTS

In [9]:
# Re-arrange orders
df_box = df_box[['Team', 'Date', 'W/L', 'Home/Away', 'Score', 'FG%', 'FGM', 
                 'FGA', '3P%', '3PM', '3PA', 'FT%', 'FTM', 'FTA', 'REB', 'OREB', 
                 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]
df_box

,Team,Date,W/L,Home/Away,Score,FG%,FGM,FGA,3P%,3PM,...,FTA,REB,OREB,DREB,AST,STL,BLK,TOV,PF,PTS
0,CLE,2018-06-08,L,Home,GSW108-85CLE,0.345,30,87,0.296,8,...,25,44,17,27,21,5,5,11,22,85
1,GSW,2018-06-08,W,Away,CLE85-108GSW,0.453,39,86,0.368,14,...,16,44,10,34,25,7,13,10,24,108
2,GSW,2018-06-06,W,Away,CLE102-110GSW,0.519,42,81,0.346,9,...,19,37,6,31,27,6,5,11,20,110
3,CLE,2018-06-06,L,Home,GSW110-102CLE,0.435,40,92,0.290,9,...,17,47,15,32,20,6,4,14,18,102
4,GSW,2018-06-03,W,Home,CLE103-122GSW,0.573,47,82,0.417,15,...,21,41,7,34,28,3,8,12,25,122
5,CLE,2018-06-03,L,Away,GSW122-103CLE,0.411,37,90,0.333,9,...,26,42,16,26,25,9,4,11,15,103
6,GSW,2018-05-31,W,Home,CLE114-124GSW,0.511,46,90,0.361,13,...,20,38,4,34,31,10,6,8,18,124
7,CLE,2018-05-31,L,Away,GSW124-114CLE,0.444,44,99,0.270,10,...,22,53,19,34,18,5,3,12,18,114
8,GSW,2018-05-28,W,Away,HOU92-101GSW,0.488,39,80,0.410,16,...,14,42,11,31,25,9,7,17,22,101
9,HOU,2018-05-28,L,Home,GSW101-92HOU,0.400,36,90,0.159,7,...,22,44,17,27,17,13,3,12,17,92


In [10]:
# Sort by 'Date'
df_box = df_box.sort_values(by=['Date'])
df_box

,Team,Date,W/L,Home/Away,Score,FG%,FGM,FGA,3P%,3PM,...,FTA,REB,OREB,DREB,AST,STL,BLK,TOV,PF,PTS
163,MIA,2018-04-14,L,Away,PHI130-103MIA,0.455,35,77,0.462,12,...,31,41,9,32,25,4,6,18,25,103
156,WAS,2018-04-14,L,Away,TOR114-106WAS,0.477,41,86,0.381,8,...,18,35,6,29,29,11,3,14,21,106
157,TOR,2018-04-14,W,Home,WAS106-114TOR,0.532,41,77,0.533,16,...,20,38,5,33,26,6,7,17,18,114
158,POR,2018-04-14,L,Home,NOP97-95POR,0.378,37,98,0.308,12,...,12,52,15,37,17,10,6,12,15,95
162,PHI,2018-04-14,W,Home,MIA103-130PHI,0.474,45,95,0.643,18,...,29,50,17,33,34,9,4,11,23,130
160,GSW,2018-04-14,W,Home,SAS92-113GSW,0.543,44,81,0.455,10,...,22,51,10,41,32,6,6,15,18,113
161,SAS,2018-04-14,L,Away,GSW113-92SAS,0.400,32,80,0.409,9,...,24,30,3,27,19,9,4,13,20,92
159,NOP,2018-04-14,W,Away,POR95-97NOP,0.477,41,86,0.333,8,...,10,49,7,42,26,8,12,15,17,97
148,UTA,2018-04-15,L,Away,OKC116-108UTA,0.471,41,87,0.393,11,...,20,42,9,33,18,10,7,14,21,108
150,MIN,2018-04-15,L,Away,HOU104-101MIN,0.438,35,80,0.348,8,...,28,47,11,36,22,3,2,14,19,101


In [11]:
# Save arranged box as .csv
df_box.to_csv(outPathArranged + dateOfCrawl + '_' + seasonYear + '_' + seasonType + '.csv', encoding='utf-8', index=False, float_format='%.3f')

## Clean-up, Pair Games, and Check Validity

In [12]:
# @param df: pandas.DataFrame
# @return pandas.DataFrame
# NaN cleaner (Numerical)
def cleanDataFrame(df):
    assert isinstance(df, pd.DataFrame), 'df needs to be a pd.DataFrame'
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].reset_index(drop=True)

# @param df: pandas.DataFrame
# @return pandas.DataFrame
# Drop objects which are NaN in Score's label (String)
def dropNanScore(df):
    index = []
    for idx, score in enumerate(df['Score']):
        if score[:3] == 'NAN' or score[:3] == 'NaN':
            index.append(idx)
    print('Number of objects dropped =', len(index))
    return df.drop(df.index[index]).reset_index(drop=True)

# @param df: pandas.DataFrame
# @return df: pandas.DataFrame
# Add opponent label to a game
def addOpponentCol(df):
    opponent = [None] * len(df['Score'])
    for idx, score in enumerate(df['Score']):
        opponent[idx] = score[:3]
    df['Opponent'] = opponent
    return df

# @param df: pandas.DataFrame
# @return df_team, df_oppo: pandas.DataFrame
# Pair two teams in a single game by searching 'Date' and 'Opponent' labels.
def pairGamePlayers(df):  
    startTime = time.time()
    invalid_idx = []
    duplicate = 0
    not_found = 0
    # Declare empty dataframe w/ columns from existing dataframe
    df_team = pd.DataFrame(columns = list(df)) # Team attributes
    df_oppo = pd.DataFrame(columns = list(df)) # Opponent attributes
    df_dupl = pd.DataFrame(columns = list(df)) # Duplicated dataframe
    for idx, date, team in zip(df.index.tolist(), df['Date'], df['Team']):
        df_oppo_searched = df.loc[lambda df: df.Date == date, :].loc[lambda df: df.Opponent == team, :]
        if len(df_oppo_searched.index.tolist()) > 1:
            duplicate += 1
            df_dupl = pd.concat([df_dupl, df_oppo_searched], ignore_index=True)
            df_oppo_searched = df_oppo_searched.iloc[0:1, :]
        if not df_oppo_searched.empty:
            df_team = pd.concat([df_team, df.iloc[idx:idx+1, :]], ignore_index=True)
            df_oppo = pd.concat([df_oppo, df_oppo_searched], ignore_index=True)
        else:
            invalid_idx.append(idx)
            not_found += 1
    
    print('Duplicate found =', duplicate)
    print('Opponent not found =', not_found)
    print('Team length = ', len(df_team.index.tolist()))
    print('Oppo length = ', len(df_oppo.index.tolist()))
    print('Execution time =', time.time() - startTime)
    return df_team, df_oppo, invalid_idx

# @param df_team, df_oppo: pandas.DataFrame
# @return df_team, df_oppo: pandas.DataFrame
# Check game validity after pairGamePlayers(df) which pairs two teams in a single game.
def checkGameValidity(df_team, df_oppo):
    startTime = time.time()
    err = 0
    invalid_idx = []
    print('Team length = ', len(df_team.index.tolist()))
    print('Oppo length = ', len(df_oppo.index.tolist()))
    for idx in df_team.index.tolist():
        if df_team.loc[idx]['Date'] != df_oppo.loc[idx]['Date'] or \
        df_team.loc[idx]['Opponent'] != df_oppo.loc[idx]['Team'] or \
        df_team.loc[idx]['W/L'] == df_oppo.loc[idx]['W/L'] or \
        df_team.loc[idx]['Home/Away'] == df_oppo.loc[idx]['Home/Away']:
            err += 1
            invalid_idx.append(idx)
    
    df_team = df_team.drop(df_team.index[invalid_idx]).reset_index(drop=True)
    df_oppo = df_oppo.drop(df_oppo.index[invalid_idx]).reset_index(drop=True)
    
    print('Number of invalid games =', err, '@', [x for x in invalid_idx])
    print('Execution time =', time.time() - startTime)
    return df_team, df_oppo, invalid_idx

In [13]:
# Remove NaN
df_box = cleanDataFrame(df_box)
df_box = dropNanScore(df_box)

# Add opponent label
df_box = addOpponentCol(df_box)

# Binary encode W/L and Home/Away
df_box['W/L'] = df_box['W/L'].map({'W':1, 'L':0})
df_box['Home/Away'] = df_box['Home/Away'].map({'Home':1, 'Away':0})

# Pair teams and opponents
df_team, df_oppo, invalid_idx = pairGamePlayers(df_box)

# Check games' validity
df_team, df_oppo, invalid_idx = checkGameValidity(df_team, df_oppo)

# Rename column: Attributes_A and Attributes_B for team and opponent, respectively
df_team = df_team.rename(columns=lambda x: x + '_A')
df_oppo = df_oppo.rename(columns=lambda x: x + '_B')

# Concatenate by column
df_output = pd.concat([df_team, df_oppo], axis=1)

Number of objects dropped = 0
Duplicate found = 0
Opponent not found = 0
Team length =  164
Oppo length =  164
Execution time = 1.0329108238220215
Team length =  164
Oppo length =  164
Number of invalid games = 0 @ []
Execution time = 0.1862478256225586


In [14]:
df_output

,Team_A,Date_A,W/L_A,Home/Away_A,Score_A,FG%_A,FGM_A,FGA_A,3P%_A,3PM_A,...,REB_B,OREB_B,DREB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PTS_B,Opponent_B
0,MIA,2018-04-14,0,0,PHI130-103MIA,0.455,35,77,0.462,12,...,50,17,33,34,9,4,11,23,130,MIA
1,WAS,2018-04-14,0,0,TOR114-106WAS,0.477,41,86,0.381,8,...,38,5,33,26,6,7,17,18,114,WAS
2,TOR,2018-04-14,1,1,WAS106-114TOR,0.532,41,77,0.533,16,...,35,6,29,29,11,3,14,21,106,TOR
3,POR,2018-04-14,0,1,NOP97-95POR,0.378,37,98,0.308,12,...,49,7,42,26,8,12,15,17,97,POR
4,PHI,2018-04-14,1,1,MIA103-130PHI,0.474,45,95,0.643,18,...,41,9,32,25,4,6,18,25,103,PHI
5,GSW,2018-04-14,1,1,SAS92-113GSW,0.543,44,81,0.455,10,...,30,3,27,19,9,4,13,20,92,GSW
6,SAS,2018-04-14,0,0,GSW113-92SAS,0.400,32,80,0.409,9,...,51,10,41,32,6,6,15,18,113,SAS
7,NOP,2018-04-14,1,0,POR95-97NOP,0.477,41,86,0.333,8,...,52,15,37,17,10,6,12,15,95,NOP
8,UTA,2018-04-15,0,0,OKC116-108UTA,0.471,41,87,0.393,11,...,46,9,37,16,8,5,17,19,116,UTA
9,MIN,2018-04-15,0,0,HOU104-101MIN,0.438,35,80,0.348,8,...,37,8,29,15,8,8,11,19,104,MIN


## Create/Update nbaGamePair.csv

In [15]:
if isAppend and os.path.exists(outFile):
    df_old = pd.read_csv(outFile)
    df_new = df_old.append(df_output, ignore_index=True)
    df_new = df_new.drop_duplicates(subset=['Team_A', 'Date_A'], keep='first')
    df_new = df_new.sort_values(by=['Date_A'])
    df_new.to_csv(outFile, encoding='utf-8', header=True, index=False, float_format='%.3f')
else:
    df_output.to_csv(outFile, encoding='utf-8', header=True, index=False, float_format='%.3f')